In [11]:
import pandas as pd

# web crawling packages
import requests
import json
from pandas import json_normalize

# from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.support.select import Select
# import chromedriver_autoinstaller
# import selenium

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

## I found a couple different ways we can crawl for both odds and unique Game and Team Id's to standardize a database if we need it.

In [2]:
# Set up the Chrome WebDriver
driver_service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=driver_service)

# set up the driver
driver.get('https://www.espn.com/college-football/odds')

# get the data elements
all_vars_list = driver.find_elements(By.ID, "topOdd") ## has the line values
open_odds_list = driver.find_elements(By.CSS_SELECTOR, "div.NIhmB.AsfGG.FuEs") ## odds for open only
odds_list = driver.find_elements(By.CSS_SELECTOR, "div.ToujM.AsfGG.FuEs") ## odds for spread, o/u, and moneyline

In [3]:
print("Odds Line Values Count", len(all_vars_list))
print("Open Odds Count", len(open_odds_list))
print("Odds Count", len(odds_list))

Odds Line Values Count 1130
Open Odds Count 332
Odds Count 572


In [37]:
# for i in odds_list:
for i in all_vars_list:
    print(i.text)
    # print(i.get_attribute("innerHTML"))
    # print(i.get_attribute("href"))

StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=126.0.6478.127); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00BBC203+27395]
	(No symbol) [0x00B53E04]
	(No symbol) [0x00A51B7F]
	(No symbol) [0x00A5FE47]
	(No symbol) [0x00A6063E]
	(No symbol) [0x00A5774B]
	(No symbol) [0x00A55A84]
	(No symbol) [0x00A588F8]
	(No symbol) [0x00A58970]
	(No symbol) [0x00A8DD91]
	(No symbol) [0x00AB399C]
	(No symbol) [0x00A88C15]
	(No symbol) [0x00AB3C34]
	(No symbol) [0x00ACCB24]
	(No symbol) [0x00AB3736]
	(No symbol) [0x00A87541]
	(No symbol) [0x00A880BD]
	GetHandleVerifier [0x00E73AB3+2876339]
	GetHandleVerifier [0x00EC7F7D+3221629]
	GetHandleVerifier [0x00C3D674+556916]
	GetHandleVerifier [0x00C4478C+585868]
	(No symbol) [0x00B5CE44]
	(No symbol) [0x00B59858]
	(No symbol) [0x00B599F7]
	(No symbol) [

## *something to keep in mind. I wonder if this page will keep historicals to refer back to. At what point should we close the off and save the odds??*

## **Another way to get the current weeks games, team logos, etc. Works very similar to the NFL use case.**
- This has game ids, odds, logos, pretty much everything we would need.

In [43]:
# make get request to this url https://site.web.api.espn.com/apis/v2/scoreboard/header?sport=football&league=college-football&region=us&lang=en&contentorigin=espn&buyWindow=1m&showAirings=buy%2Clive%2Creplay&showZipLookup=true&tz=America%2FNew_York
# get the data from the response. I found this url by inspecting the network tab in the browser

url = 'https://site.web.api.espn.com/apis/v2/scoreboard/header?sport=football&league=college-football&region=us&lang=en&contentorigin=espn&buyWindow=1m&showAirings=buy%2Clive%2Creplay&showZipLookup=true&tz=America%2FNew_York'

response = requests.get(url)

# response_text = json.loads(response.text)
# espn_api_df = json_normalize(response_text['sports'], ['leagues', 'events'])
# print(len(espn_api_df))

In [44]:
response.json()

{'code': 404}

In [39]:
response_text = json.loads(response.text)
espn_api_df = json_normalize(response_text['sports'], ['leagues', 'events'])
print(len(espn_api_df))
col_list = espn_api_df.columns
print(col_list)

23
Index(['id', 'uid', 'date', 'timeValid', 'recent', 'name', 'shortName',
       'notes', 'links', 'gamecastAvailable',
       ...
       'fullStatus.displayClock', 'fullStatus.period', 'fullStatus.type.id',
       'fullStatus.type.name', 'fullStatus.type.state',
       'fullStatus.type.completed', 'fullStatus.type.description',
       'fullStatus.type.detail', 'fullStatus.type.shortDetail', 'airings'],
      dtype='object', length=208)


In [17]:
# espn_api_df.to_csv('espn_api_data.csv', index=False)

In [25]:
norm1 = json_normalize(espn_api_df['competitors'])
norm1.columns = ['home', 'away']
norm1

,home,away
0,"{'id': '52', 'uid': 's:20~l:23~t:52', 'type': ...","{'id': '59', 'uid': 's:20~l:23~t:59', 'type': ..."
1,"{'id': '2717', 'uid': 's:20~l:23~t:2717', 'typ...","{'id': '152', 'uid': 's:20~l:23~t:152', 'type'..."
2,"{'id': '93', 'uid': 's:20~l:23~t:93', 'type': ...","{'id': '142', 'uid': 's:20~l:23~t:142', 'type'..."
3,"{'id': '2815', 'uid': 's:20~l:23~t:2815', 'typ...","{'id': '2305', 'uid': 's:20~l:23~t:2305', 'typ..."
4,"{'id': '253', 'uid': 's:20~l:23~t:253', 'type'...","{'id': '254', 'uid': 's:20~l:23~t:254', 'type'..."
5,"{'id': '218', 'uid': 's:20~l:23~t:218', 'type'...","{'id': '201', 'uid': 's:20~l:23~t:201', 'type'..."
6,"{'id': '228', 'uid': 's:20~l:23~t:228', 'type'...","{'id': '61', 'uid': 's:20~l:23~t:61', 'type': ..."
7,"{'id': '213', 'uid': 's:20~l:23~t:213', 'type'...","{'id': '277', 'uid': 's:20~l:23~t:277', 'type'..."
8,"{'id': '2287', 'uid': 's:20~l:23~t:2287', 'typ...","{'id': '2294', 'uid': 's:20~l:23~t:2294', 'typ..."
9,"{'id': '236', 'uid': 's:20~l:23~t:236', 'type'...","{'id': '2633', 'uid': 's:20~l:23~t:2633', 'typ..."


In [28]:
home_meta = json_normalize(norm1['home'])
away_meta = json_normalize(norm1['away'])
away_meta.head()

,id,uid,type,order,homeAway,winner,displayName,name,abbreviation,location,color,alternateColor,score,links,group,record,logo,logoDark,records,rank
0,59,s:20~l:23~t:59,team,0,home,False,Georgia Tech Yellow Jackets,Georgia Tech,GT,Georgia Tech,003057,b3a369,,"[{'language': 'en-US', 'rel': ['clubhouse', 'd...",1,0-0,https://a.espncdn.com/i/teamlogos/ncaa/500/59.png,https://a.espncdn.com/i/teamlogos/ncaa/500-dar...,"[{'type': 'home', 'summary': '0-0-0', 'display...",NaN
1,152,s:20~l:23~t:152,team,0,home,False,NC State Wolfpack,NC State,NCSU,NC State,cc0000,ffffff,,"[{'language': 'en-US', 'rel': ['clubhouse', 'd...",1,0-0,https://a.espncdn.com/i/teamlogos/ncaa/500/152...,https://a.espncdn.com/i/teamlogos/ncaa/500-dar...,"[{'type': 'home', 'summary': '0-0-0', 'display...",24.0
2,142,s:20~l:23~t:142,team,0,home,False,Missouri Tigers,Missouri,MIZ,Missouri,f1b82d,000000,,"[{'language': 'en-US', 'rel': ['clubhouse', 'd...",8,0-0,https://a.espncdn.com/i/teamlogos/ncaa/500/142...,https://a.espncdn.com/i/teamlogos/ncaa/500-dar...,"[{'type': 'home', 'summary': '0-0-0', 'display...",11.0
3,2305,s:20~l:23~t:2305,team,0,home,False,Kansas Jayhawks,Kansas,KU,Kansas,0051ba,e8000d,,"[{'language': 'en-US', 'rel': ['clubhouse', 'd...",4,0-0,https://a.espncdn.com/i/teamlogos/ncaa/500/230...,https://a.espncdn.com/i/teamlogos/ncaa/500-dar...,"[{'type': 'home', 'summary': '0-0-0', 'display...",22.0
4,254,s:20~l:23~t:254,team,0,home,False,Utah Utes,Utah,UTAH,Utah,ea002a,ffffff,,"[{'language': 'en-US', 'rel': ['clubhouse', 'd...",4,0-0,https://a.espncdn.com/i/teamlogos/ncaa/500/254...,https://a.espncdn.com/i/teamlogos/ncaa/500-dar...,"[{'type': 'home', 'summary': '0-0-0', 'display...",12.0


In [57]:
logos = pd.concat([home_meta[['logo', 'name']], away_meta[['logo', 'name']]], axis=1)
logos.columns = ['home_logo', 'home_name', 'away_logo', 'away_name']
logos

,home_logo,home_name,away_logo,away_name
0,https://a.espncdn.com/i/teamlogos/ncaa/500/52.png,Florida St,https://a.espncdn.com/i/teamlogos/ncaa/500/59.png,Georgia Tech
1,https://a.espncdn.com/i/teamlogos/ncaa/500/271...,W Carolina,https://a.espncdn.com/i/teamlogos/ncaa/500/152...,NC State
2,https://a.espncdn.com/i/teamlogos/ncaa/500/93.png,Murray St,https://a.espncdn.com/i/teamlogos/ncaa/500/142...,Missouri
3,https://a.espncdn.com/i/teamlogos/ncaa/500/281...,Lindenwood,https://a.espncdn.com/i/teamlogos/ncaa/500/230...,Kansas
4,https://a.espncdn.com/i/teamlogos/ncaa/500/253...,Southern Utah,https://a.espncdn.com/i/teamlogos/ncaa/500/254...,Utah
5,https://a.espncdn.com/i/teamlogos/ncaa/500/218...,Temple,https://a.espncdn.com/i/teamlogos/ncaa/500/201...,Oklahoma
6,https://a.espncdn.com/i/teamlogos/ncaa/500/228...,Clemson,https://a.espncdn.com/i/teamlogos/ncaa/500/61.png,Georgia
7,https://a.espncdn.com/i/teamlogos/ncaa/500/213...,Penn State,https://a.espncdn.com/i/teamlogos/ncaa/500/277...,West Virginia
8,https://a.espncdn.com/i/teamlogos/ncaa/500/228...,Illinois St,https://a.espncdn.com/i/teamlogos/ncaa/500/229...,Iowa
9,https://a.espncdn.com/i/teamlogos/ncaa/500/236...,Chattanooga,https://a.espncdn.com/i/teamlogos/ncaa/500/263...,Tennessee


In [55]:
model_ready = espn_api_df[['id', 'name', 'odds.awayTeamOdds.moneyLine', 'odds.awayTeamOdds.spreadOdds'
                        , 'odds.homeTeamOdds.moneyLine', 'odds.homeTeamOdds.spreadOdds'
                        , 'odds.pointSpread.home.close.line', 'odds.pointSpread.away.close.line'
                        , 'odds.total.over.close.line', 'odds.total.over.close.odds', 'odds.total.under.close.odds']]

model_ready.rename(columns={'odds.awayTeamOdds.moneyLine': 'away_money_line'
                            , 'odds.awayTeamOdds.spreadOdds': 'away_point_spread_line'
                            , 'odds.homeTeamOdds.moneyLine': 'home_money_line'
                            , 'odds.homeTeamOdds.spreadOdds': 'home_point_spread_line'
                            , 'odds.pointSpread.home.close.line': 'home_point_spread'
                            , 'odds.pointSpread.away.close.line': 'away_point_spread'
                            , 'odds.total.over.close.line': 'over_under'
                            , 'odds.total.over.close.odds': 'over_line'
                            , 'odds.total.under.close.odds': 'under_line'}, inplace=True)
# fill na with 0
model_ready.fillna(0, inplace=True)

model_ready['home_point_spread'] = pd.to_numeric(model_ready['home_point_spread'])
model_ready['away_point_spread'] = pd.to_numeric(model_ready['away_point_spread'])
model_ready['over_line'] = pd.to_numeric(model_ready['over_line'])
model_ready['under_line'] = pd.to_numeric(model_ready['under_line'])

# replace the 'o' string with nothing for over_under column and make numeric
model_ready['over_under'] = model_ready['over_under'].str.replace('o', '')
model_ready['over_under'] = pd.to_numeric(model_ready['over_under'])
model_ready.fillna(0, inplace=True)
model_ready.head()

model_ready_logo = pd.concat([model_ready, logos], axis=1)
model_ready_logo.head()

C:\Users\rober\AppData\Local\Temp\ipykernel_21380\2255616135.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_ready.rename(columns={'odds.awayTeamOdds.moneyLine': 'away_money_line'
C:\Users\rober\AppData\Local\Temp\ipykernel_21380\2255616135.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_ready.fillna(0, inplace=True)
C:\Users\rober\AppData\Local\Temp\ipykernel_21380\2255616135.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

,id,name,away_money_line,away_point_spread_line,home_money_line,home_point_spread_line,home_point_spread,away_point_spread,over_under,over_line,under_line,logo,logo
0,401635525,Florida State Seminoles at Georgia Tech Yellow...,-450.0,-105.0,360.0,-115.0,11.5,-11.5,56.5,-110,-110,https://a.espncdn.com/i/teamlogos/ncaa/500/52.png,https://a.espncdn.com/i/teamlogos/ncaa/500/59.png
1,401634299,Western Carolina Catamounts at NC State Wolfpack,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,https://a.espncdn.com/i/teamlogos/ncaa/500/271...,https://a.espncdn.com/i/teamlogos/ncaa/500/152...
2,401628327,Murray State Racers at Missouri Tigers,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,https://a.espncdn.com/i/teamlogos/ncaa/500/93.png,https://a.espncdn.com/i/teamlogos/ncaa/500/142...
3,401636606,Lindenwood Lions at Kansas Jayhawks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,https://a.espncdn.com/i/teamlogos/ncaa/500/281...,https://a.espncdn.com/i/teamlogos/ncaa/500/230...
4,401636607,Southern Utah Thunderbirds at Utah Utes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,https://a.espncdn.com/i/teamlogos/ncaa/500/253...,https://a.espncdn.com/i/teamlogos/ncaa/500/254...


In [51]:
model_ready.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      23 non-null     object 
 1   name                    23 non-null     object 
 2   away_money_line         23 non-null     float64
 3   away_point_spread_line  23 non-null     float64
 4   home_money_line         23 non-null     float64
 5   home_point_spread_line  23 non-null     float64
 6   home_point_spread       23 non-null     float64
 7   away_point_spread       23 non-null     float64
 8   over_under              23 non-null     object 
 9   over_line               23 non-null     object 
 10  under_line              23 non-null     object 
dtypes: float64(6), object(5)
memory usage: 2.1+ KB


## Teams

In [90]:
# set up the driver
driver.get('https://www.espn.com/college-football/teams')
# //*[@id="fittPageContainer"]/div[3]/div/div/div[1]/div[2]/div[1]/div[1]/div[2]/div[1]/div/section/a
# get the data elements
teams_list = driver.find_elements(By.XPATH, '//*[@class="ContentList__Item"]/div/section/a') ## has the line values

In [91]:
for i in teams_list:
    print(i.text)
    print(i.get_attribute("href"))


https://www.espn.com/college-football/team/_/id/103/boston-college-eagles

https://www.espn.com/college-football/team/_/id/228/clemson-tigers

https://www.espn.com/college-football/team/_/id/150/duke-blue-devils

https://www.espn.com/college-football/team/_/id/52/florida-state-seminoles

https://www.espn.com/college-football/team/_/id/59/georgia-tech-yellow-jackets

https://www.espn.com/college-football/team/_/id/97/louisville-cardinals

https://www.espn.com/college-football/team/_/id/2390/miami-hurricanes

https://www.espn.com/college-football/team/_/id/152/nc-state-wolfpack

https://www.espn.com/college-football/team/_/id/153/north-carolina-tar-heels

https://www.espn.com/college-football/team/_/id/221/pittsburgh-panthers

https://www.espn.com/college-football/team/_/id/183/syracuse-orange

https://www.espn.com/college-football/team/_/id/258/virginia-cavaliers

https://www.espn.com/college-football/team/_/id/259/virginia-tech-hokies

https://www.espn.com/college-football/team/_/id/1